#Малютин Илья ИУ5-64Б РК №2 Вариант - 9
####Задание.
Для заданного набора данных [https://www.kaggle.com/rubenssjr/brasilian-houses-to-rent (файл houses_to_rent.csv)] постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы:
  - Линейная/логистическая регрессия
  - Градиентный бустинг


 Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

###Загрузка данных


In [6]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Загрузка данных
data = pd.read_csv("houses_to_rent.csv")
print(data.head())

   Unnamed: 0  city  area  rooms  bathroom  parking spaces floor     animal  \
0           0     1   240      3         3               4     -      acept   
1           1     0    64      2         1               1    10      acept   
2           2     1   443      5         5               4     3      acept   
3           3     1    73      2         2               1    12      acept   
4           4     1    19      1         1               0     -  not acept   

       furniture      hoa rent amount property tax fire insurance     total  
0      furnished      R$0     R$8,000      R$1,000          R$121   R$9,121  
1  not furnished    R$540       R$820        R$122           R$11   R$1,493  
2      furnished  R$4,172     R$7,000      R$1,417           R$89  R$12,680  
3  not furnished    R$700     R$1,250        R$150           R$16   R$2,116  
4  not furnished      R$0     R$1,200         R$41           R$16   R$1,257  


###Предобработка данных, преобразование категориальных признаков, заполнение пропусков

In [10]:
# Предобработка данных: удаление столбцов с большим количеством пропусков
# Удаление лишних символов, замена запятых на точки
data = data.dropna(thresh=len(data) * 0.5, axis=1)
data = data.applymap(lambda x: str(x).replace('R$', '') if isinstance(x, str) else x)
data = data.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)


# Преобразование категориальных признаков методом one-hot кодирования
categorical_features = data.select_dtypes(include=["object"]).columns
encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
encoded_features = encoder.fit_transform(data[categorical_features])

# Заполнение пропусков числовых признаков средним значением
numerical_features = data.select_dtypes(include=["number"]).columns
imputer = SimpleImputer(strategy="mean")
imputed_features = imputer.fit_transform(data[numerical_features])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Масштабирование числовых признаков, соединение обработанных признаков и разделение на обучающую и тестовую выборки

In [11]:
# Масштабирование числовых признаков
scaler = StandardScaler()
scaled_features = scaler.fit_transform(imputed_features)

# Соединение обработанных признаков
X = np.hstack([encoded_features, scaled_features])
y = data["rent amount"]  # Замена на целевую переменную

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


###Построение модели

- Линейная регрессия

In [12]:
# Построение моделей

# Линейная регрессия
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

- Градиентный бустинг

In [13]:
# Градиентный бустинг (регрессия)
gb_reg = GradientBoostingRegressor()
gb_reg.fit(X_train, y_train)

GradientBoostingRegressor()

###Оценка моделей

In [14]:
# Оценка моделей

# Метрики качества:
# - Среднеквадратичная ошибка (RMSE)
# - R-квадрат (R²)

# Линейная регрессия
y_pred_linear = linear_reg.predict(X_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))
r2_linear = r2_score(y_test, y_pred_linear)

# Градиентный бустинг (регрессия)
y_pred_gb = gb_reg.predict(X_test)
rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))
r2_gb = r2_score(y_test, y_pred_gb)

# Вывод
print("Линейная регрессия:")
print(f"RMSE: {rmse_linear}")
print(f"R^2: {r2_linear}")

print("\nГрадиентный бустинг (регрессия):")
print(f"RMSE: {rmse_gb}")
print(f"R^2: {r2_gb}")


Линейная регрессия:
RMSE: 76.30117122437271
R^2: 0.8166000107992691

Градиентный бустинг (регрессия):
RMSE: 92.74688554061218
R^2: 0.7290210431731733


Я использовал метрики RMSE (Среднеквадратичная ошибка) и R² (коэффициент детерминации) для оценки качества моделей по следующим причинам:
1. RMSE является прямой мерой качества предсказаний, показывая среднюю разницу между фактическими и прогнозируемыми значениями в тех же единицах измерения, что и сами значения. Это делает интерпретацию ошибки более интуитивно понятной и в большей степени наглядной для не специалистов.
2. R² (коэффициент детерминации) отображает долю дисперсии зависимой переменной, которая объясняется моделью. Эта метрика позволяет понять, насколько хорошо модель объясняет изменчивость данных, предоставляя относительное представление о качестве модели.

Эти метрики используются, потому что они дополняют друг друга: RMSE предоставляет информацию о точности предсказаний в абсолютных величинах, а R² помогает оценить объяснительную мощность модели.

###Вывод
Линейная регрессия имеет более низкий RMSE и более высокий R^2, чем градиентный бустинг, что говорит о том, что линейная регрессия является более точной моделью для предсказания арендной платы за дом в этом наборе данных.